In [1]:
import numpy as np 
import pandas as pd
import gc
import os
import glob
import pyarrow
pd.set_option('display.max_columns', None)

In [2]:
# Already loaded on kaggle
calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
#train_val_o = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
sell_prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')
train_val_o = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')


# Taking sample to prevent kernel
#np.random.seed(42)
#sample_ids = np.random.choice(train_val_o['id'].unique(), size=1000, replace=False)
#train_val = train_val_o[train_val_o['id'].isin(sample_ids)]
train_val = train_val_o

In [3]:
# EDA steps
print("-- Missing values in calendar --")
print(calendar.isnull().sum())

print("\n-- Data types (info) --")
calendar.info()

print("\n-- Descriptive statistics --")
print(calendar.describe(include='all'))

print("-- Missing values in sell_prices --")
print(sell_prices.isnull().sum())

print("\n-- Data types (info) --")
sell_prices.info()

print("\n-- Descriptive stats for sell_price --")
print(sell_prices["sell_price"].describe())

print("Unique item IDs:", train_val["item_id"].nunique())
print("Unique store IDs:", train_val["store_id"].nunique())
print("Unique dept IDs:", train_val["dept_id"].nunique())
print("Unique cat IDs:", train_val["cat_id"].nunique())
print("Unique state IDs:", train_val["state_id"].nunique())

-- Missing values in calendar --
date               0
wm_yr_wk           0
weekday            0
wday               0
month              0
year               0
d                  0
event_name_1    1807
event_type_1    1807
event_name_2    1964
event_type_2    1964
snap_CA            0
snap_TX            0
snap_WI            0
dtype: int64

-- Data types (info) --
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          1969 non-null   object
 1   wm_yr_wk      1969 non-null   int64 
 2   weekday       1969 non-null   object
 3   wday          1969 non-null   int64 
 4   month         1969 non-null   int64 
 5   year          1969 non-null   int64 
 6   d             1969 non-null   object
 7   event_name_1  162 non-null    object
 8   event_type_1  162 non-null    object
 9   event_name_2  5 non-null      object
 10  event_type_2  5 non

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64

-- Data types (info) --
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB

-- Descriptive stats for sell_price --
count    6.841121e+06
mean     4.410952e+00
std      3.408814e+00
min      1.000000e-02
25%      2.180000e+00
50%      3.470000e+00
75%      5.840000e+00
max      1.073200e+02
Name: sell_price, dtype: float64
Unique item IDs: 3049
Unique store IDs: 10
Unique dept IDs: 7
Unique cat IDs: 3
Unique state IDs: 3


In [4]:
# Melt the train_val to merge with other datasets
sales_eval = pd.melt(train_val, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()
print("Shape of train_val after melt:", sales_eval.shape)

# Merge with calendar data
df = pd.merge(sales_eval, calendar, on='d', how='left')
print("Shape after merged with calendar data:", df.shape)

Shape of train_val after melt: (59181090, 8)
Shape after merged with calendar data: (59181090, 21)


In [5]:
# Downcast the data for easy and fast processing
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()

    for i, t in enumerate(types):
        col = cols[i]

        if 'int' in str(t):
            if df[col].min() > np.iinfo(np.int8).min and df[col].max() < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif df[col].min() > np.iinfo(np.int16).min and df[col].max() < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif df[col].min() > np.iinfo(np.int32).min and df[col].max() < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)

        elif 'float' in str(t):
            if df[col].min() > np.finfo(np.float16).min and df[col].max() < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif df[col].min() > np.finfo(np.float32).min and df[col].max() < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

        elif t == object:
            if col == 'date':
                df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
            else:
                df[col] = df[col].astype('category')

    return df

df = downcast(df)

# Convert date column back to datetime
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'])

print(df.shape)
print('After memory reduction:')
print(df.info())

(59181090, 21)
After memory reduction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 21 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            category      
 1   item_id       category      
 2   dept_id       category      
 3   cat_id        category      
 4   store_id      category      
 5   state_id      category      
 6   d             category      
 7   sold          int16         
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       category      
 11  wday          int8          
 12  month         int8          
 13  year          int16         
 14  event_name_1  category      
 15  event_type_1  category      
 16  event_name_2  category      
 17  event_type_2  category      
 18  snap_CA       int8          
 19  snap_TX       int8          
 20  snap_WI       int8          
dtypes: category(12), datetime64[ns](1), int16(3), int8(5)
memor

In [6]:
# Merge with price data
df = pd.merge(df, sell_prices, on=['store_id','item_id','wm_yr_wk'], how='left')
print("Shape after merged with price data:", df.shape)

df.head()

Shape after merged with price data: (59181090, 22)


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


In [8]:
del calendar, train_val_o, sell_prices, train_val  # example
gc.collect()

0

In [10]:
dfs = []
for _, sub_df in df.groupby('store_id'):
    sub_df = sub_df.sort_values('date').copy()
    
    sub_df['weekofyear'] = sub_df['date'].dt.isocalendar().week
    sub_df['dayofmonth'] = sub_df['date'].dt.day
    sub_df['day_of_year'] = sub_df['date'].dt.dayofyear
    sub_df['weekend'] = sub_df['date'].dt.weekday.isin([5, 6]).astype(np.uint8)
    sub_df['is_month_start'] = sub_df['date'].dt.is_month_start.astype(np.uint8)
    sub_df['is_month_end'] = sub_df['date'].dt.is_month_end.astype(np.uint8)
    sub_df['is_quarter_start'] = sub_df['date'].dt.is_quarter_start.astype(np.uint8)
    sub_df['is_quarter_end'] = sub_df['date'].dt.is_quarter_end.astype(np.uint8)
    sub_df['is_event'] = sub_df['event_name_1'].notnull().astype(np.uint8)
    sub_df['wday'] = sub_df['date'].dt.weekday + 1

    dfs.append(sub_df)

df = pd.concat(dfs).reset_index(drop=True)
del dfs, sub_df
gc.collect()
print(df.shape)

/tmp/ipykernel_31/310429233.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for _, sub_df in df.groupby('store_id'):


(59181090, 31)


In [11]:
# Event/Holiday Features

# Step 1: Ensure datetime and sorting
df = df.sort_values('date')
df['date'] = pd.to_datetime(df['date'])

# Step 2: Create event DataFrame
events = df[df['event_name_1'].notnull()][['date']].copy()
events = events.sort_values('date')
events = events.rename(columns={'date': 'event_date'})

# Step 3: Use merge_asof for previous and next events
df = pd.merge_asof(df, events, left_on='date', right_on='event_date', direction='backward')
df['days_since_event'] = (df['date'] - df['event_date']).dt.days

df = pd.merge_asof(df, events, left_on='date', right_on='event_date', direction='forward', suffixes=('', '_next'))
df['days_until_event'] = (df['event_date_next'] - df['date']).dt.days

# Data Cleanup
df.drop(columns=['event_date', 'event_date_next'], inplace=True)
print(df.shape)
df.to_feather('/kaggle/working/df.feather')

(59181090, 33)


In [12]:
# Price Features

df['price_max'] = df.groupby('item_id')['sell_price'].transform('max')
df['price_min'] = df.groupby('item_id')['sell_price'].transform('min')
df['price_std'] = df.groupby('item_id')['sell_price'].transform('std')
df['price_norm'] = df['sell_price'] / df['price_max']
df = df.sort_values(['item_id', 'date'])
df['prev_price'] = df.groupby('item_id')['sell_price'].shift(1)
df['price_momentum'] = df['sell_price'] / df['prev_price']
df['prev_month_price'] = df.groupby('item_id')['sell_price'].shift(28)  # assuming daily data & 28-day months
df['price_change_from_last_month'] = (df['sell_price'] - df['prev_month_price']).fillna(0)
df['price_avg'] = df.groupby('item_id')['sell_price'].transform('mean')
df['discount_flag'] = (df['sell_price'] < df['price_avg']).astype(int)
df.drop(columns=['prev_price', 'prev_month_price', 'price_avg'], inplace=True)
print(df.shape)
df.to_feather('/kaggle/working/df.feather')

/tmp/ipykernel_31/2266465746.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['price_max'] = df.groupby('item_id')['sell_price'].transform('max')
/tmp/ipykernel_31/2266465746.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['price_min'] = df.groupby('item_id')['sell_price'].transform('min')
/tmp/ipykernel_31/2266465746.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['price_std'] = df.groupby('item_id')

(59181090, 40)


In [13]:
# Promotion/SNAP Features

# Ensure date column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Vectorized SNAP flag creation
conditions = [
    df['store_id'].str.startswith('CA'),
    df['store_id'].str.startswith('TX'),
    df['store_id'].str.startswith('WI')
]
choices = [df['snap_CA'], df['snap_TX'], df['snap_WI']]
df['snap_active'] = np.select(conditions, choices, default=0)

# Sort values
df = df.sort_values(['store_id', 'date']).reset_index(drop=True)

# Create helper: last SNAP active date using forward fill logic
df['snap_date'] = df['date'].where(df['snap_active'] == 1)

# Forward fill within store groups
df['last_snap_date'] = df.groupby('store_id')['snap_date'].ffill()

# Compute days since last SNAP active
df['days_since_snap'] = (df['date'] - df['last_snap_date']).dt.days
df.loc[df['snap_active'] == 1, 'days_since_snap'] = 0

df['cumulative_snap_days'] = df.groupby('store_id')['snap_active'].cumsum()

# Create 'snap_date_future' column where SNAP is active
df['snap_date_future'] = df['date'].where(df['snap_active'] == 1)

# Reverse the DataFrame within each group to apply forward-fill "backwards"
df['snap_date_future'] = (
    df.iloc[::-1]
    .groupby('store_id')['snap_date_future']
    .ffill()
    .iloc[::-1]
)

# Calculate days until next SNAP date
df['days_until_next_snap'] = (df['snap_date_future'] - df['date']).dt.days
df.loc[df['snap_active'] == 1, 'days_until_next_snap'] = 0

# Cleanup
df.drop(columns=['snap_date_future','snap_date','last_snap_date','cumulative_snap_days'], inplace=True)
print(df.shape)
df.to_feather('/kaggle/working/df.feather')

/tmp/ipykernel_31/676198016.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['last_snap_date'] = df.groupby('store_id')['snap_date'].ffill()
/tmp/ipykernel_31/676198016.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['cumulative_snap_days'] = df.groupby('store_id')['snap_active'].cumsum()
/tmp/ipykernel_31/676198016.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby('store_id')['snap_date_future

(59181090, 43)


In [14]:
#Lag features
lags = [1, 7, 14, 28]

for lag in lags:
    df[f'sales_lag_{lag}'] = df.groupby('id')['sold'].shift(lag)

print(df.shape)
df.to_feather('/kaggle/working/df.feather')

/tmp/ipykernel_31/2648118170.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f'sales_lag_{lag}'] = df.groupby('id')['sold'].shift(lag)
/tmp/ipykernel_31/2648118170.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f'sales_lag_{lag}'] = df.groupby('id')['sold'].shift(lag)
/tmp/ipykernel_31/2648118170.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f'sales_lag_{lag}'] = df.groupby('id')['sold'].shift(lag

(59181090, 47)


In [15]:
# Downcast the data for easy and fast processing
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()

    for i, t in enumerate(types):
        col = cols[i]

        if 'int' in str(t):
            if df[col].min() > np.iinfo(np.int8).min and df[col].max() < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif df[col].min() > np.iinfo(np.int16).min and df[col].max() < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif df[col].min() > np.iinfo(np.int32).min and df[col].max() < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)

        elif 'float' in str(t):
            if df[col].min() > np.finfo(np.float16).min and df[col].max() < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif df[col].min() > np.finfo(np.float32).min and df[col].max() < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

        elif t == object:
            if col == 'date':
                df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
            else:
                df[col] = df[col].astype('category')

    return df

df = downcast(df)

# Convert date column back to datetime
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'])
df.to_feather('/kaggle/working/df.feather')

print(df.shape)
print('After memory reduction:')
print(df.info())

(59181090, 47)
After memory reduction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 47 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   id                            category      
 1   item_id                       category      
 2   dept_id                       category      
 3   cat_id                        category      
 4   store_id                      category      
 5   state_id                      category      
 6   d                             category      
 7   sold                          int16         
 8   date                          datetime64[ns]
 9   wm_yr_wk                      int16         
 10  weekday                       category      
 11  wday                          int8          
 12  month                         int8          
 13  year                          int16         
 14  event_name_1                  category   

In [16]:
folder_path = "/kaggle/working/rolling_chunks"

# Find all files in the folder
files = glob.glob(os.path.join(folder_path, "*"))

# Remove each file
for file in files:
    os.remove(file)

In [17]:
# Rolling Window Features

# Ensure data is sorted correctly
df = df.sort_values(['id', 'date']).copy()
output_path = '/kaggle/working/rolling_chunks'
os.makedirs(output_path, exist_ok=True)

id_list = df['id'].unique()
batch_size = 4000

for i in range(0, len(id_list), batch_size):
    batch_ids = id_list[i:i+batch_size]
    chunk = df[df['id'].isin(batch_ids)].copy()
    chunk = chunk.sort_values(['id', 'date'])

    # Safe rolling and expanding using groupby().transform()
    chunk['rolling_mean_7'] = chunk.groupby('id')['sold'].transform(lambda x: x.shift(1).rolling(7).mean()).astype(np.float32)
    chunk['rolling_std_7'] = chunk.groupby('id')['sold'].transform(lambda x: x.shift(1).rolling(7).std()).astype(np.float32)
    chunk['rolling_mean_14'] = chunk.groupby('id')['sold'].transform(lambda x: x.shift(1).rolling(14).mean()).astype(np.float32)
    chunk['rolling_std_14'] = chunk.groupby('id')['sold'].transform(lambda x: x.shift(1).rolling(14).std()).astype(np.float32)
    chunk['expanding_mean'] = chunk.groupby('id')['sold'].transform(lambda x: x.shift(1).expanding().mean()).astype(np.float32)
    chunk['rolling_min_7'] = chunk.groupby('id')['sold'].transform(lambda x: x.shift(1).rolling(7).min()).astype(np.float32)
    chunk['rolling_max_7'] = chunk.groupby('id')['sold'].transform(lambda x: x.shift(1).rolling(7).max()).astype(np.float32)

    # Save batch to disk
    chunk.to_feather(f"{output_path}/rolling_batch_{i}.feather")
    del chunk
    gc.collect()

/tmp/ipykernel_31/2789390266.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk['rolling_mean_7'] = chunk.groupby('id')['sold'].transform(lambda x: x.shift(1).rolling(7).mean()).astype(np.float32)
/tmp/ipykernel_31/2789390266.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chunk['rolling_std_7'] = chunk.groupby('id')['sold'].transform(lambda x: x.shift(1).rolling(7).std()).astype(np.float32)
/tmp/ipykernel_31/2789390266.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or obse

In [39]:
import pyarrow as pa
import pyarrow.parquet as pq

input_folder = '/kaggle/working/rolling_chunks'
feather_files = sorted(glob.glob(f"{input_folder}/*.feather"))
output_file = '/kaggle/working/df_rolling.parquet'

batch_size = 3
buffer = []
writer = None

for i, file in enumerate(feather_files):
    print(f"📄 Reading {file} ({i+1}/{len(feather_files)})")
    chunk = pd.read_feather(file)
    buffer.append(chunk)

    if len(buffer) == batch_size or i == len(feather_files) - 1:
        df_out = pd.concat(buffer, ignore_index=True)
        table = pa.Table.from_pandas(df_out)

        # Initialize writer on first write
        if writer is None:
            writer = pq.ParquetWriter(output_file, table.schema, compression='snappy')

        writer.write_table(table)
        print(f"✅ Written batch ending with file {i+1}")

        buffer.clear()

# Close the writer properly
if writer is not None:
    writer.close()
    print(f"🎉 All data written to: {output_file}")

📄 Reading /kaggle/working/rolling_chunks/rolling_batch_0.feather (1/8)
📄 Reading /kaggle/working/rolling_chunks/rolling_batch_12000.feather (2/8)
📄 Reading /kaggle/working/rolling_chunks/rolling_batch_16000.feather (3/8)
✅ Written batch ending with file 3
📄 Reading /kaggle/working/rolling_chunks/rolling_batch_20000.feather (4/8)
📄 Reading /kaggle/working/rolling_chunks/rolling_batch_24000.feather (5/8)
📄 Reading /kaggle/working/rolling_chunks/rolling_batch_28000.feather (6/8)
✅ Written batch ending with file 6
📄 Reading /kaggle/working/rolling_chunks/rolling_batch_4000.feather (7/8)
📄 Reading /kaggle/working/rolling_chunks/rolling_batch_8000.feather (8/8)
✅ Written batch ending with file 8
🎉 All data written to: /kaggle/working/df_rolling.parquet


In [3]:
# Load the Parquet file
df = pd.read_parquet('/kaggle/working/df_rolling.parquet')

# Save as Feather
df.to_feather('/kaggle/working/df_rolling.feather')

print(df.shape)

(59181090, 54)


In [4]:
# Interaction features

df['snap_event'] = df['snap_active'] * df['is_event'].notna().astype(int)
df['price_event'] = df['discount_flag'] * df['is_event'].notna().astype(int)
df['weekend_event'] = df['weekend'] * df['is_event'].notna().astype(int)

print(df.shape)
df.to_feather('/kaggle/working/df.feather')

(59181090, 57)


In [5]:
# Cumulative/Expanding Features

df['cumulative_sales'] = df.groupby('id')['sold'].cumsum()
df['cumulative_mean_sales'] = df.groupby('id')['sold'].transform(lambda x: x.expanding().mean())

def days_since_last_sale(x):
    out = []
    last_day = -1
    for i, val in enumerate(x):
        if val > 0:
            last_day = i
        out.append(i - last_day if last_day != -1 else np.nan)
    return out

fallback_days = (df['date'] - pd.to_datetime('2011-01-29')).dt.days
df['days_since_last_sale'] = df.groupby('id')['sold'].transform(days_since_last_sale)
df['days_since_last_sale'] = df['days_since_last_sale'].fillna(fallback_days)

print(df.shape)
df.to_feather('/kaggle/working/df.feather')

/tmp/ipykernel_212/2883972013.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['cumulative_sales'] = df.groupby('id')['sold'].cumsum()
/tmp/ipykernel_212/2883972013.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['cumulative_mean_sales'] = df.groupby('id')['sold'].transform(lambda x: x.expanding().mean())
/tmp/ipykernel_212/2883972013.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['days_since_last_s

(59181090, 60)


In [6]:
# Hierarchical Aggregated Features

# Aggregated average sales at dept level
df['dept_sales'] = df.groupby(['dept_id', 'd'])['sold'].transform('mean')

# Aggregated average sales at store level
df['store_sales'] = df.groupby(['store_id', 'd'])['sold'].transform('mean')

# Aggregated average sales at state level
df['state_sales'] = df.groupby(['state_id', 'd'])['sold'].transform('mean')

# Aggregated average sales at category level
df['cat_sales'] = df.groupby(['cat_id', 'd'])['sold'].transform('mean')

df['rolling_mean_store_dept_7'] = (
    df.groupby(['store_id', 'dept_id'])['sold']
      .transform(lambda x: x.shift(1).rolling(7).mean())
)

# Add a 7-day lag of that rolling mean
df['rolling_mean_store_dept_7_lag_7'] = (
    df.groupby(['store_id', 'dept_id'])['rolling_mean_store_dept_7']
      .shift(7)
)

print(df.shape)
df.to_feather('/kaggle/working/df.feather')

/tmp/ipykernel_212/242494185.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['dept_sales'] = df.groupby(['dept_id', 'd'])['sold'].transform('mean')
/tmp/ipykernel_212/242494185.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['store_sales'] = df.groupby(['store_id', 'd'])['sold'].transform('mean')
/tmp/ipykernel_212/242494185.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['state_sales'] = df.groupby

(59181090, 66)


In [7]:
# Lag on Engineered Features

df['rolling_mean_7_lag_7'] = df.groupby('id')['rolling_mean_7'].shift(7)
df['rolling_mean_14_lag_14'] = df.groupby('id')['rolling_mean_14'].shift(14)
df['rolling_mean_change'] = df['rolling_mean_7'] - df['rolling_mean_7_lag_7']

print(df.shape)
df.to_feather('/kaggle/working/df.feather')

/tmp/ipykernel_212/1950850414.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['rolling_mean_7_lag_7'] = df.groupby('id')['rolling_mean_7'].shift(7)
/tmp/ipykernel_212/1950850414.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['rolling_mean_14_lag_14'] = df.groupby('id')['rolling_mean_14'].shift(14)


(59181090, 69)


In [8]:
# Downcast the data for easy and fast processing
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()

    for i, t in enumerate(types):
        col = cols[i]

        if 'int' in str(t):
            if df[col].min() > np.iinfo(np.int8).min and df[col].max() < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif df[col].min() > np.iinfo(np.int16).min and df[col].max() < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif df[col].min() > np.iinfo(np.int32).min and df[col].max() < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.int64)

        elif 'float' in str(t):
            if df[col].min() > np.finfo(np.float16).min and df[col].max() < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif df[col].min() > np.finfo(np.float32).min and df[col].max() < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

        elif t == object:
            if col == 'date':
                df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
            else:
                df[col] = df[col].astype('category')

    return df

df = downcast(df)

# Convert date column back to datetime
df['date'] = df['date'].astype(str)
df['date'] = pd.to_datetime(df['date'])
df.to_feather('/kaggle/working/df.feather')

print(df.shape)
print('After memory reduction:')
print(df.info())

(59181090, 69)
After memory reduction:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59181090 entries, 0 to 59181089
Data columns (total 69 columns):
 #   Column                           Dtype         
---  ------                           -----         
 0   id                               category      
 1   item_id                          category      
 2   dept_id                          category      
 3   cat_id                           category      
 4   store_id                         category      
 5   state_id                         category      
 6   d                                category      
 7   sold                             int16         
 8   date                             datetime64[ns]
 9   wm_yr_wk                         int16         
 10  weekday                          category      
 11  wday                             int8          
 12  month                            int8          
 13  year                             int16        

In [2]:
df = pd.read_feather("/kaggle/working/df.feather")
df.shape

(59181090, 69)

In [3]:
# Dropping data for first 28 days to take care for NA values
df = df[df['date'] > '2011-02-25'].reset_index(drop=True)

print(df.shape)
df.to_feather('/kaggle/working/df.feather')

(58327370, 69)


In [4]:
cols_to_replace = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']

for col in cols_to_replace:
    if col in df.columns:
        if pd.api.types.is_categorical_dtype(df[col]):
            # Add "no" to category list if it's not already present
            if "no" not in df[col].cat.categories:
                df[col] = df[col].cat.add_categories("no")
        # Now fill the NaNs
        df[col] = df[col].fillna("no")

/tmp/ipykernel_262/2616146203.py:5: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df[col]):


In [5]:
price_na_cols = ['sell_price', 'price_norm', 'price_momentum']

for col in price_na_cols:
    df[f'{col}_is_missing'] = df[col].isna().astype(int)
    df[col] = df[col].fillna(0)

In [6]:
forward_looking_cols = ['days_until_event', 'days_until_next_snap']

for col in forward_looking_cols:
    df[f'{col}_is_missing'] = df[col].isna().astype(int)
    df[col] = df[col].fillna(999)

In [7]:
print(df.shape)
df.to_feather('/kaggle/working/df.feather')

(58327370, 74)
